In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import glob
from datetime import datetime,date
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymongo
import csv
import json
import re
import os
import psycopg2
from urllib.parse import urlencode, urlparse, parse_qsl
from config import port_id
from config import username
from config import database
from config import pwd

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

# Connecting to Postgres

In [2]:
#Connecting to Postgres
conn = psycopg2.connect(
        host = hostname,
        dbname = database,
        user = username,
        password = pwd, 
        port = port_id)
print("database opened successfully")

database opened successfully


# Download from Top 10

In [3]:
#Downloading the data from Postgres
df = pd.read_sql('SELECT * FROM top10', conn)
df.head()

,city,province_state,country_region,lat,long,confirmed_cases,cases_per_day,last_update,people_fully_vaccinated,coordinates,real_values,predicted_values
0,None,Channel Islands,United Kingdom,49.372300,-2.364400,0.0,8.0,2021-04-02,5205505,"[49.3723, -2.3644]",0.0,0.0
1,Loving,Texas,US,31.849476,-103.581857,1.0,1.0,2021-01-30,7182375,"[31.84947612, -103.5818571]",1.0,1.0
2,Harding,New Mexico,US,35.857516,-103.822094,8.0,8.0,2021-01-30,7182375,"[35.85751555, -103.82209379999999]",8.0,8.0
3,Petroleum,Montana,US,47.118246,-108.250888,11.0,10.0,2021-01-30,7182375,"[47.118246299999996, -108.2508876]",11.0,11.0
4,Hinsdale,Colorado,US,37.822474,-107.300434,16.0,16.0,2021-01-30,7182375,"[37.82247444, -107.3004345]",16.0,16.0


In [4]:
city_data_df = df
city_data_df.head(10)

,city,province_state,country_region,lat,long,confirmed_cases,cases_per_day,last_update,people_fully_vaccinated,coordinates,real_values,predicted_values
0,None,Channel Islands,United Kingdom,49.372300,-2.364400,0.0,8.0,2021-04-02,5205505,"[49.3723, -2.3644]",0.0,0.0
1,Loving,Texas,US,31.849476,-103.581857,1.0,1.0,2021-01-30,7182375,"[31.84947612, -103.5818571]",1.0,1.0
2,Harding,New Mexico,US,35.857516,-103.822094,8.0,8.0,2021-01-30,7182375,"[35.85751555, -103.82209379999999]",8.0,8.0
3,Petroleum,Montana,US,47.118246,-108.250888,11.0,10.0,2021-01-30,7182375,"[47.118246299999996, -108.2508876]",11.0,11.0
4,Hinsdale,Colorado,US,37.822474,-107.300434,16.0,16.0,2021-01-30,7182375,"[37.82247444, -107.3004345]",16.0,16.0
5,Skagway,Alaska,US,59.561500,-135.333775,16.0,16.0,2021-01-30,7182375,"[59.56149996, -135.33377480000001]",16.0,16.0
6,Blaine,Nebraska,US,41.913117,-99.976778,19.0,19.0,2021-01-30,7182375,"[41.91311716, -99.97677845]",19.0,19.0
7,None,Montserrat,United Kingdom,16.742498,-62.187366,20.0,6.0,2021-02-25,736037,"[16.742498, -62.187366000000004]",20.0,20.0
8,Wheeler,Oregon,US,44.726982,-120.028143,22.0,21.0,2021-01-30,7182375,"[44.72698162, -120.02814270000002]",22.0,21.9
9,Arthur,Nebraska,US,41.568961,-101.695956,22.0,22.0,2021-01-30,7182375,"[41.56896142, -101.6959559]",22.0,22.0


In [5]:
city_data_df.dtypes

city                        object
province_state              object
country_region              object
lat                        float64
long                       float64
confirmed_cases            float64
cases_per_day              float64
last_update                 object
people_fully_vaccinated      int64
coordinates                 object
real_values                float64
predicted_values           float64
dtype: object

In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df
preferred_cities_df.head(10)

,city,province_state,country_region,lat,long,confirmed_cases,cases_per_day,last_update,people_fully_vaccinated,coordinates,real_values,predicted_values
0,None,Channel Islands,United Kingdom,49.372300,-2.364400,0.0,8.0,2021-04-02,5205505,"[49.3723, -2.3644]",0.0,0.0
1,Loving,Texas,US,31.849476,-103.581857,1.0,1.0,2021-01-30,7182375,"[31.84947612, -103.5818571]",1.0,1.0
2,Harding,New Mexico,US,35.857516,-103.822094,8.0,8.0,2021-01-30,7182375,"[35.85751555, -103.82209379999999]",8.0,8.0
3,Petroleum,Montana,US,47.118246,-108.250888,11.0,10.0,2021-01-30,7182375,"[47.118246299999996, -108.2508876]",11.0,11.0
4,Hinsdale,Colorado,US,37.822474,-107.300434,16.0,16.0,2021-01-30,7182375,"[37.82247444, -107.3004345]",16.0,16.0
5,Skagway,Alaska,US,59.561500,-135.333775,16.0,16.0,2021-01-30,7182375,"[59.56149996, -135.33377480000001]",16.0,16.0
6,Blaine,Nebraska,US,41.913117,-99.976778,19.0,19.0,2021-01-30,7182375,"[41.91311716, -99.97677845]",19.0,19.0
7,None,Montserrat,United Kingdom,16.742498,-62.187366,20.0,6.0,2021-02-25,736037,"[16.742498, -62.187366000000004]",20.0,20.0
8,Wheeler,Oregon,US,44.726982,-120.028143,22.0,21.0,2021-01-30,7182375,"[44.72698162, -120.02814270000002]",22.0,21.9
9,Arthur,Nebraska,US,41.568961,-101.695956,22.0,22.0,2021-01-30,7182375,"[41.56896142, -101.6959559]",22.0,22.0


In [7]:
# 5b. Create a new column "Hotel Name"
# hotel_df["Hotel Name"] = preferred_cities_df
hotel_df = preferred_cities_df[["country_region", "city", "province_state", "lat", "long", "confirmed_cases", "people_fully_vaccinated", "predicted_values"]].copy()
hotel_df

,country_region,city,province_state,lat,long,confirmed_cases,people_fully_vaccinated,predicted_values
0,United Kingdom,None,Channel Islands,49.372300,-2.364400,0.0,5205505,0.0
1,US,Loving,Texas,31.849476,-103.581857,1.0,7182375,1.0
2,US,Harding,New Mexico,35.857516,-103.822094,8.0,7182375,8.0
3,US,Petroleum,Montana,47.118246,-108.250888,11.0,7182375,11.0
4,US,Hinsdale,Colorado,37.822474,-107.300434,16.0,7182375,16.0
5,US,Skagway,Alaska,59.561500,-135.333775,16.0,7182375,16.0
6,US,Blaine,Nebraska,41.913117,-99.976778,19.0,7182375,19.0
7,United Kingdom,None,Montserrat,16.742498,-62.187366,20.0,736037,20.0
8,US,Wheeler,Oregon,44.726982,-120.028143,22.0,7182375,21.9
9,US,Arthur,Nebraska,41.568961,-101.695956,22.0,7182375,22.0


In [8]:
# Add Hotel Name
hotel_df["Hotel Name"] = ""
hotel_df["Rating"] = ""
hotel_df

,country_region,city,province_state,lat,long,confirmed_cases,people_fully_vaccinated,predicted_values,Hotel Name,Rating
0,United Kingdom,None,Channel Islands,49.372300,-2.364400,0.0,5205505,0.0,,
1,US,Loving,Texas,31.849476,-103.581857,1.0,7182375,1.0,,
2,US,Harding,New Mexico,35.857516,-103.822094,8.0,7182375,8.0,,
3,US,Petroleum,Montana,47.118246,-108.250888,11.0,7182375,11.0,,
4,US,Hinsdale,Colorado,37.822474,-107.300434,16.0,7182375,16.0,,
5,US,Skagway,Alaska,59.561500,-135.333775,16.0,7182375,16.0,,
6,US,Blaine,Nebraska,41.913117,-99.976778,19.0,7182375,19.0,,
7,United Kingdom,None,Montserrat,16.742498,-62.187366,20.0,736037,20.0,,
8,US,Wheeler,Oregon,44.726982,-120.028143,22.0,7182375,21.9,,
9,US,Arthur,Nebraska,41.568961,-101.695956,22.0,7182375,22.0,,


In [9]:
# 6a. Set parameters to search for hotels with 10000 meters.
params = {
    "radius": 10000,
    "type": "lodging",
    "key": g_key,
    "fields": "name, formatted_address, price_level, rating"
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["lat"]
    lng = row["long"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    print(hotels)
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        hotel_df.loc[index, "Rating"] = hotels ["results"][1]["rating"]
    
    except (IndexError):
        print("Hotel not found... skipping.")

{'html_attributions': [], 'next_page_token': 'Aap_uEDixd03OQJkk8EqzBSYtyvhiOobVQ1iLE5LVLU2-rA5TwNoQxdI_3U0DOvrTGZjO02JjKBO_WbzZPf9bDSS05qgDhBG3sIquvEDjM1bQRWyq0oRXOSGpuFK87anCe5kqyK9rRz66WEFay7uX4TzPLwyvoL5qqvhZ9zGG3cxzH4vbFBquXPwqvS1tCDTw__-Yju05-QpTGdXUkUJ1E3Pdtz2OQt7EsefuLy2pYqsegz2gqBE955IvpyRjaEvK8J_FffenIcPIbzxJwQcZ8ClEJGGg2GnCXrWcxRPe3BY1tMF9ejYbrGXmgaocYRl8hDgImC5NYS2575g6EwM47fNgHtfkP38sRjeShdX0O-cHBbTCAmmWF9GOc-fOp1Q8Jsk5MVuZXmouXXkBHeJ3pdWWCAyRHjiyLVUQtTY2WnT7S77hBltafjPYP8', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 49.4284328, 'lng': -2.3611394}, 'viewport': {'northeast': {'lat': 49.42960758029149, 'lng': -2.359810119708498}, 'southwest': {'lat': 49.4269096197085, 'lng': -2.362508080291501}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Stocks Hotel', 'photos':

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 31.82345100000001, 'lng': -103.6376245}, 'viewport': {'northeast': {'lat': 31.8251565802915, 'lng': -103.63493365}, 'southwest': {'lat': 31.8224586197085, 'lng': -103.63852145}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Target Hospitality-Station Lodge Mentone', 'photos': [{'height': 3024, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/100938987894738637558">Ismael Zapata</a>'], 'photo_reference': 'Aap_uEDjcIek8Pr3y_xr8vBFx-NIwLg-lw9LBAGwjKhr-HsopsX6EZvyV4sKS3zET098Qi_D9DlytIPDZykjKONDtFYHAawCLCYXKotuELARxS9SZ1lOnHsAhQQaEE2HTxfWbjDDpu-dFLcjHTCUN0thqPI__k30VaRzjUjl_xAwFmxuDPcm', 'width': 4032}], 'place_id': 'ChIJR2V4pYfO5IYRrUrie8bYN_k', 'plus_code': {'compound_code': 'R9F6+9X Mento

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 41.9388904, 'lng': -99.86484790000002}, 'viewport': {'northeast': {'lat': 41.94020773029149, 'lng': -99.86349926970851}, 'southwest': {'lat': 41.93750976970849, 'lng': -99.86619723029152}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/camping-71.png', 'icon_background_color': '#4DB546', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/camping_pinlet', 'name': 'Uncle Bucks Campground', 'place_id': 'ChIJgeVkz1HYnIcRbCYg14RdhwI', 'plus_code': {'compound_code': 'W4QP+H3 Brewster, NE, USA', 'global_code': '86H2W4QP+H3'}, 'reference': 'ChIJgeVkz1HYnIcRbCYg14RdhwI', 'scope': 'GOOGLE', 'types': ['campground', 'lodging', 'park', 'point_of_interest', 'establishment'], 'vicinity': 'Brewster'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 41.940371, 'lng': -99.8696946}, 'viewport': {'northeast': {'lat': 41.9417199802915

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 44.79299289999999, 'lng': -120.0015707}, 'viewport': {'northeast': {'lat': 44.79444823029149, 'lng': -120.0003838697085}, 'southwest': {'lat': 44.79175026970849, 'lng': -120.0030818302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/camping-71.png', 'icon_background_color': '#4DB546', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/camping_pinlet', 'name': 'Service Creek Campground and Boat Launch', 'photos': [{'height': 3464, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/115278294293084135580">Jalet Farrell</a>'], 'photo_reference': 'Aap_uEDmD_e-cLw7JCy8p2ygfH3DuUxJi_6CoEPxc7sHfqaD6rJ3jQeXMtt3P1UwdHLDlRXRxNXx7OodiUeccjGIqCpDQ8FaGxWp-tHh4FcwhdDc7r-T6ssmHrvF4IZNthmpK6i7ZpwbzIAVNYkIAl5UJzDLe8vBoRhb5t7OO7lk37-2-Dnj', 'width': 4618}], 'place_id': 'ChIJ7zYnY75tvFQRTlTjYI-KZVw', 'plus_code': {'compound_code': 

In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,country_region,city,province_state,lat,long,confirmed_cases,people_fully_vaccinated,predicted_values,Hotel Name,Rating
1,US,Loving,Texas,31.849476,-103.581857,1.0,7182375,1.0,Target Hospitality-Station Lodge Mentone,3.7
2,US,Harding,New Mexico,35.857516,-103.822094,8.0,7182375,8.0,,
3,US,Petroleum,Montana,47.118246,-108.250888,11.0,7182375,11.0,,
4,US,Hinsdale,Colorado,37.822474,-107.300434,16.0,7182375,16.0,Lost Trail Campground,5
5,US,Skagway,Alaska,59.561500,-135.333775,16.0,7182375,16.0,Chilkoot Trail Outpost,4.8
6,US,Blaine,Nebraska,41.913117,-99.976778,19.0,7182375,19.0,Uncle Bucks Campground,5
8,US,Wheeler,Oregon,44.726982,-120.028143,22.0,7182375,21.9,Service Creek Campground and Boat Launch,4
9,US,Arthur,Nebraska,41.568961,-101.695956,22.0,7182375,22.0,Logge Ron,


In [11]:
# Convert to NaN then drop
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df

,country_region,city,province_state,lat,long,confirmed_cases,people_fully_vaccinated,predicted_values,Hotel Name,Rating
0,United Kingdom,None,Channel Islands,49.372300,-2.364400,0.0,5205505,0.0,Stocks Hotel,4.3
1,US,Loving,Texas,31.849476,-103.581857,1.0,7182375,1.0,Target Hospitality-Station Lodge Mentone,3.7
2,US,Harding,New Mexico,35.857516,-103.822094,8.0,7182375,8.0,NaN,
3,US,Petroleum,Montana,47.118246,-108.250888,11.0,7182375,11.0,NaN,
4,US,Hinsdale,Colorado,37.822474,-107.300434,16.0,7182375,16.0,Lost Trail Campground,5
5,US,Skagway,Alaska,59.561500,-135.333775,16.0,7182375,16.0,Chilkoot Trail Outpost,4.8
6,US,Blaine,Nebraska,41.913117,-99.976778,19.0,7182375,19.0,Uncle Bucks Campground,5
7,United Kingdom,None,Montserrat,16.742498,-62.187366,20.0,736037,20.0,SeaView Suite,4.3
8,US,Wheeler,Oregon,44.726982,-120.028143,22.0,7182375,21.9,Service Creek Campground and Boat Launch,4
9,US,Arthur,Nebraska,41.568961,-101.695956,22.0,7182375,22.0,Logge Ron,


In [12]:
# Drop
hotel_df = hotel_df.dropna(subset=["Hotel Name"])
hotel_df

,country_region,city,province_state,lat,long,confirmed_cases,people_fully_vaccinated,predicted_values,Hotel Name,Rating
0,United Kingdom,None,Channel Islands,49.372300,-2.364400,0.0,5205505,0.0,Stocks Hotel,4.3
1,US,Loving,Texas,31.849476,-103.581857,1.0,7182375,1.0,Target Hospitality-Station Lodge Mentone,3.7
4,US,Hinsdale,Colorado,37.822474,-107.300434,16.0,7182375,16.0,Lost Trail Campground,5
5,US,Skagway,Alaska,59.561500,-135.333775,16.0,7182375,16.0,Chilkoot Trail Outpost,4.8
6,US,Blaine,Nebraska,41.913117,-99.976778,19.0,7182375,19.0,Uncle Bucks Campground,5
7,United Kingdom,None,Montserrat,16.742498,-62.187366,20.0,736037,20.0,SeaView Suite,4.3
8,US,Wheeler,Oregon,44.726982,-120.028143,22.0,7182375,21.9,Service Creek Campground and Boat Launch,4
9,US,Arthur,Nebraska,41.568961,-101.695956,22.0,7182375,22.0,Logge Ron,


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "top_10_hotel_list.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file)
hotel_df.head()

,country_region,city,province_state,lat,long,confirmed_cases,people_fully_vaccinated,predicted_values,Hotel Name,Rating
0,United Kingdom,None,Channel Islands,49.372300,-2.364400,0.0,5205505,0.0,Stocks Hotel,4.3
1,US,Loving,Texas,31.849476,-103.581857,1.0,7182375,1.0,Target Hospitality-Station Lodge Mentone,3.7
4,US,Hinsdale,Colorado,37.822474,-107.300434,16.0,7182375,16.0,Lost Trail Campground,5
5,US,Skagway,Alaska,59.561500,-135.333775,16.0,7182375,16.0,Chilkoot Trail Outpost,4.8
6,US,Blaine,Nebraska,41.913117,-99.976778,19.0,7182375,19.0,Uncle Bucks Campground,5


In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Rating</dt><dd>{Rating}</dd>
<dt>City Name</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country_region}</dd>
<dt>Confirmed Covid Cases</dt><dd>{confirmed_cases}</dd>
<dt>Projected Covid Cases</dt><dd>{predicted_values}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["lat", "long"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["lat", "long"]]
fig = gmaps.figure(center=(16.74249, -62.187366), zoom_level=0.8)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
from ipywidgets.embed import embed_minimal_html

In [17]:
embed_minimal_html('export.html', views=[fig])